In [5]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
import highway_env  
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
import numpy as np


STUDY TO REALIZE:

HYPERPARAMETERS 
TEST WITH TEACHERS ENVIROMENT 

In [2]:
env = gym.make("racetrack-v0",render_mode="rgb_array")

config = {
    "observation": {
        "type": "OccupancyGrid",
        "vehicles_count": 10,
        "features": ['presence', 'on_road'],
        "grid_size": [[-18, 18], [-18, 18]],
        "grid_step": [3, 3],
        "as_image": False,
        "align_to_vehicle_axes": True
    },
    "action": {
        "type": "ContinuousAction",
        "longitudinal": False,
        "lateral": True
    },
    "simulation_frequency": 15,
    "policy_frequency": 5,
    "duration": 300,
    "collision_reward": -2,
    "lane_centering_cost": 2,
    "action_reward": -0.3,
    "controlled_vehicles": 1,
    "other_vehicles": 10,
    "screen_width": 600,
    "screen_height": 600,
    "centering_position": [0.5, 0.5],
    "scaling": 7,
    "show_trajectories": False,
    "render_agent": True,
    "offscreen_rendering": False
}
env.unwrapped.configure(config)
env.reset()

(array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

gamma 0.6 0.8 0.99 
lr 5e-4 and 1e-5

In [3]:
batch_size = 64
model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
    batch_size=batch_size,
    n_epochs=10,
    learning_rate=5e-4,
    gamma=0.8,
    verbose=2,
)

# Train the model
model.learn(total_timesteps=int(5e4))
model.save("racetrack_ppo/model")
del model

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/vitoriano/anaconda3/envs/RL/lib/python3.9/site-packages/stable_baselines3/common/policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 802      |
|    ep_rew_mean     | 24       |
| time/              |          |
|    fps             | 37       |
|    iterations      | 1        |
|    time_elapsed    | 53       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.04e+03    |
|    ep_rew_mean          | 23.6        |
| time/                   |             |
|    fps                  | 39          |
|    iterations           | 2           |
|    time_elapsed         | 104         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012742138 |
|    clip_fraction        | 0.0506      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.39       |
|    explained_variance   | -0.297      |
|    learning_rate        | 0.

In [6]:
# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.8 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=int(5e4), log_interval=10)
model.save("racetrack_ddpg/model")
vec_env = model.get_env()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 389      |
|    ep_rew_mean     | 48       |
| time/              |          |
|    episodes        | 10       |
|    fps             | 37       |
|    time_elapsed    | 104      |
|    total_timesteps | 3894     |
| train/             |          |
|    actor_loss      | -1.69    |
|    critic_loss     | 0.0029   |
|    learning_rate   | 0.001    |
|    n_updates       | 3793     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 333      |
|    ep_rew_mean     | 115      |
| time/              |          |
|    episodes        | 20       |
|    fps             | 36       |
|    time_elapsed    | 183      |
|    total_timesteps | 6662     |
| train/             |          |
|    actor_loss      | -9.77    |
|    critic_loss     

/home/vitoriano/anaconda3/envs/RL/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'racetrack_ddpg' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [9]:
env.config["duration"] = 30

/home/vitoriano/anaconda3/envs/RL/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.config to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.config` for environment variables or `env.get_wrapper_attr('config')` that will search the reminding wrappers.
  logger.warn(


In [10]:
model = PPO.load("racetrack_ppo/model", env=env)

env = RecordVideo(
    env, video_folder="racetrack_ppo/videos", episode_trigger=lambda e: True
)
env.unwrapped.set_record_video_wrapper(env)

for video in range(1):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/vitoriano/anaconda3/envs/RL/lib/python3.9/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ppo/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ddpg/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ddpg/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ddpg/videos/rl-video-episode-1.mp4
Moviepy - Building video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ppo/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ppo/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ppo/videos/rl-video-episode-0.mp4


In [8]:
model = DDPG.load("racetrack_ddpg/model")

env = RecordVideo(
    env, video_folder="racetrack_ddpg/videos", episode_trigger=lambda e: True
)
env.unwrapped.set_record_video_wrapper(env)

for video in range(1):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()

/home/vitoriano/anaconda3/envs/RL/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 2.32GB > 2.21GB
  warnings.warn(


Moviepy - Building video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ddpg/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ddpg/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/racetrack_ddpg/videos/rl-video-episode-0.mp4


### Testing PPO in intersection env


In [11]:
env = gym.make("roundabout-v0",render_mode="rgb_array")
env.configure({
    "observation": {
        "type": "OccupancyGrid",
        "vehicles_count": 10,
        "features": ['presence', 'on_road'],
        "grid_size": [[-18, 18], [-18, 18]],
        "grid_step": [3, 3],
        "as_image": False,
        "align_to_vehicle_axes": True
    },
    "action": {
        "type": "ContinuousAction",
        "longitudinal": False,
        "lateral": True
    }
})
env.config["duration"] = 30
env.reset()

/home/vitoriano/anaconda3/envs/RL/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
/home/vitoriano/anaconda3/envs/RL/lib/python3.9/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.config to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.config` for environment variables or `env.get_wrapper_attr('config')` that will search the reminding wrappers.
  logger.warn(


(array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,

In [12]:
model = PPO.load("racetrack_ppo/model",env=env)

env = RecordVideo(
    env, video_folder="roundanbout_ppo/videos", episode_trigger=lambda e: True
)
env.unwrapped.set_record_video_wrapper(env)

for video in range(1):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Moviepy - Building video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/roundanbout_ppo/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/roundanbout_ppo/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/vitoriano/Documents/ws-vscode/projet_highway_to_heaven/racetrack/roundanbout_ppo/videos/rl-video-episode-0.mp4
